In [ ]:
import math
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle
import skimage
import pickle as pkl
import cv2
import tensorflow.python.platform
from keras.preprocessing import sequence
from collections import Counter

In [ ]:
#Give csv of the dataset for which feature extraction has to be performed
df = pd.read_csv('/home/neha/sem8/AMP/assignment-2/vqa assignment/VQAMed2018Valid/VQAMed2018Valid-QA.csv',sep='\t',header=None)

In [ ]:
df=df[[0,1,2]]
df.head(5)

In [ ]:
df.columns=['Serialno','imageno','question']

In [ ]:
# to remove inputs where answers span across multiple columns  
boolmat=[]
for i in range(0,len(df)):
    boolmat.append(df['question'].iloc[i][len(df['question'].iloc[i])-1]=='?')
df=df[boolmat]

In [ ]:
len(df)


In [ ]:
#path to VGG model 
vgg_path = './vqa assignment/model/vgg16-20160129.tfmodel'
#path to directory that contains images
img_path='./vqa assignment/VQAMed2018Valid/VQAMed2018Valid-images'
def crop_image(x, target_height=227, target_width=227, as_float=True):
    image = cv2.imread(x)
    if as_float:
        image = image.astype(np.float32)

    if len(image.shape) == 2:
        image = np.tile(image[:,:,None], 3)
    elif len(image.shape) == 4:
        image = image[:,:,:,0]

    height, width, rgb = image.shape
    if width == height:
        resized_image = cv2.resize(image, (target_height,target_width))

    elif height < width:
        resized_image = cv2.resize(image, (int(width * float(target_height)/height), target_width))
        cropping_length = int((resized_image.shape[1] - target_height) / 2)
        resized_image = resized_image[:,cropping_length:resized_image.shape[1] - cropping_length]

    else:
        resized_image = cv2.resize(image, (target_height, int(height * float(target_width) / width)))
        cropping_length = int((resized_image.shape[0] - target_width) / 2)
        resized_image = resized_image[cropping_length:resized_image.shape[0] - cropping_length,:]

    return cv2.resize(resized_image, (target_height, target_width))

def read_image(path):
    img = crop_image(path, target_height=224, target_width=224)
    if img.shape[2] == 4:
        img = img[:,:,:3]
    img = img[None, ...]
    return img

tf.reset_default_graph()
with open(vgg_path,'rb') as f:
        fileContent = f.read()
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(fileContent)
final_array_test = []
images = tf.placeholder("float32", [1, 224, 224, 3])
tf.import_graph_def(graph_def, input_map={"images":images})
graph = tf.get_default_graph()
    
with tf.Session() as sess:    
    for i in df.imageno:
        feat = read_image(img_path+"/"+i+".jpg")
        #Here activations extracted from the last fully connected layers are used as features
        fc7 = sess.run(graph.get_tensor_by_name("import/Relu_1:0"), feed_dict={images:feat})      
        fc7=np.squeeze(fc7)
        final_array_test.append(fc7)



In [ ]:
len(final_array_test)

In [ ]:
#Saving the features extracted for the images
import pickle
pickle.dump(final_array_test, open( "image_feature_train", "wb" ) )